In [1]:
#Imports
import numpy as np
from pandas.io.parsers import read_csv
import matplotlib.pyplot as plt
from matplotlib.axes import Axes
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
import math
import scipy.optimize as opt
from sklearn.preprocessing import PolynomialFeatures
from scipy.io import loadmat
from scipy.optimize import minimize

import displayData as disp
import checkNNGradients as grad

In [2]:
def func_coste_reg(Thetas, X, Y, lmb):
    m = np.shape(X)[0]
    return cost_funct(Y, X, m) + regularizacion(Thetas, lmb, m)

def sigmoid_funct(z):
    return 1.0 / (1.0 + np.exp(-z))

def cost_funct(Y, g, m):
    J = np.sum(-1.0*Y* np.log(g) -1.0*(1 - Y)* np.log(1-g))
    return J/m

def regularizacion(Thetas, lmb, m):
    suma = 0
    for i in Thetas:
        i = i[:,1:]
        suma += (np.sum(i**2))
    
    return (lmb/(2*m))*suma


In [3]:
def forward_propagation(X, theta1, theta2, m, Y):
    
    #Capa entrada asignamos la X con los unos incluidos
    a1 = np.hstack([np.ones([m, 1]), X])

    #capa intermedia (hidden) calculamos las ecuaciones de la anterior, aplicamos la sigmoide e incluimos los unos de la neurona 0 
    z2 = np.dot(a1, theta1.T)
    a2 = np.hstack([np.ones([m, 1]), sigmoid_funct(z2)])

    #Capa salida calcuamos las ecuaciones con theta2 y aplicamos la sigmoide, nos devuelve la matriz de salida 5000x10
    z3 = np.dot(a2, theta2.T)
    
    a3 = sigmoid_funct(z3)
    
    return a1, z2, a2, z3, a3

In [4]:
#Genera los pesos aleatorios
def RandomWeights(entradas, salidas, ini):
    Theta = np.random.uniform(-ini, ini, size = (entradas+1, salidas))
    return Theta

In [5]:

def backprop(params_rn, num_entradas, num_ocultas, num_etiquetas, X, Y, reg):

    #Recostruimos las Thetas
    Theta1 = np.reshape(params_rn[:num_ocultas *(num_entradas + 1)],(num_ocultas, (num_entradas+1)))
    Theta2 = np.reshape(params_rn[num_ocultas*(num_entradas + 1): ], (num_etiquetas,(num_ocultas+1)))
    
    m = X.shape[0]    
    y_onehot = Y
    
    a1, z2, a2, z3, a3 = forward_propagation(X, Theta1, Theta2, m, y_onehot)
    
    #Calculamos el coste
    coste = func_coste_reg([Theta1, Theta2], a3, y_onehot, reg)

    d3 = a3 - y_onehot
    d2 = np.matmul(Theta2.T,d3.T)*(a2*(1-a2)).T
    
    #Calculamos los gradientes no regularizados
    G1 = np.matmul(d2[1:,:], a1)/m
    G2 = np.matmul(d3.T,a2)/m
    
    #Calculamos los gradientes regularizados
    G1 = G1 + ((reg*1.0)/m)*np.insert(Theta1[:, 1:], 0, 0, axis = 1)
    G2 = G2 + ((reg*1.0)/m)*np.insert(Theta2[:, 1:], 0, 0, axis = 1)

    #Ponemos los gradientes en forma de lista
    gradientes = np.concatenate((G1, G2), axis = None)
    
    return coste, gradientes

In [6]:
def optimize_backprop_and_check (num_entradas, num_ocultas, num_etiquetas, reg, X, y_onehot, laps, Y, ini):
    
    #Inicializamos los pesos y los ponemos en forma de lista
    Theta1 = RandomWeights(num_entradas, num_ocultas, ini)
    Theta2 = RandomWeights(num_ocultas, num_etiquetas, ini)
    pesos = np.concatenate((Theta1, Theta2), axis=None)
    
    #Optimizamos
    out = opt.minimize(fun = backprop, x0 = pesos, args = (num_entradas, num_ocultas, num_etiquetas, X, y_onehot, reg), method='TNC', jac = True, options = {'maxiter': laps})

    #Reconstruimos las Thetas
    Theta1 = out.x[:(num_ocultas*(num_entradas+1))].reshape(num_ocultas,(num_entradas+1))
    Theta2 = out.x[(num_ocultas*(num_entradas+1)):].reshape(num_etiquetas,(num_ocultas+1))
    
    m = X.shape[0]
    #Hacemos el forward propagation
    a1, z2, a2, z3, a3 = forward_propagation(X, Theta1, Theta2, m, y_onehot)
    
    #Sacamos los aciertos
    indexes = np.argmax(a3, axis=1)
    acc = (np.sum(indexes == (Y))/m)*100
    
    #print(acc)
    return acc

In [7]:
def main():
    
    data = loadmat ("ex4data1.mat")

    #almacenamos los datos leídos en X e y
    X = data['X']
    m = X.shape[0]
    #X = np.hstack([np.ones([m, 1]), X])
    y = data['y']
    y = np.ravel(y) 
    num_labels = 10
    num_entries = np.shape(X)[1]
    num_hiden_layers = 25

    #print(X.shape)
    #disp.displayImage(X[4])
    
    #Ponemos la y en forma onehot
    y = y-1
    y_onehot = np.zeros((m, num_labels))  # 5000 x 10
    
    for i in range(m):
        y_onehot[i][y[i]] = 1
    
    weights = loadmat("ex4weights.mat")
    theta1, theta2 = weights["Theta1"], weights["Theta2"]
    reg = 1
    a1, z2, a2, z3, a3 = forward_propagation(X, theta1, theta2, m, y_onehot)
    
    #print("Coste no regulado: ",cost_funct(y_onehot, a3, m))
    #print("Coste regulado: ",func_coste_reg([theta1, theta2], a3,y_onehot, reg))
    
    #Check del gradiente con reg
    #print(grad.checkNNGradients(backprop, reg))
    
    lambdas = np.linspace(0.01, 5, 40)
    for i in lambdas:
        acc = optimize_backprop_and_check(num_entries, num_hiden_layers, num_labels, i, X, y_onehot, 70, y, 0.12)
        print("lambda =", i, "aciertos:", acc, "%")
    print("\n")

    #return acc

In [ ]:
main()

lambda = 0.01 aciertos: 91.36 %
lambda = 0.13794871794871796 aciertos: 93.34 %
lambda = 0.2658974358974359 aciertos: 91.8 %
lambda = 0.39384615384615385 aciertos: 89.53999999999999 %
lambda = 0.5217948717948718 aciertos: 94.06 %
lambda = 0.6497435897435898 aciertos: 93.04 %
lambda = 0.7776923076923077 aciertos: 93.14 %
lambda = 0.9056410256410257 aciertos: 92.14 %
lambda = 1.0335897435897436 aciertos: 92.52 %
lambda = 1.1615384615384616 aciertos: 92.80000000000001 %
lambda = 1.2894871794871796 aciertos: 89.98 %
lambda = 1.4174358974358976 aciertos: 94.17999999999999 %
lambda = 1.5453846153846154 aciertos: 91.97999999999999 %
lambda = 1.6733333333333333 aciertos: 93.5 %
lambda = 1.8012820512820513 aciertos: 92.46 %
lambda = 1.9292307692307693 aciertos: 93.10000000000001 %
lambda = 2.057179487179487 aciertos: 89.64 %
lambda = 2.185128205128205 aciertos: 94.04 %
lambda = 2.313076923076923 aciertos: 92.32000000000001 %
lambda = 2.441025641025641 aciertos: 93.0 %
lambda = 2.568974358974359 

In [ ]:
num_iter = 10
acc = 0
for i in range(0, num_iter):
    main()
    #acc = acc + main()
#acc = acc/num_iter
#print(acc, "\n\n")

In [ ]:
num_iter = 250
